In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import math
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime, date, time
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.preprocessing import label_binarize
from sklearn import datasets
%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/mauri/Documents/Important IH Files/lab-cleaning-categorical-data/files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [3]:
df_num = df.select_dtypes(np.number)

In [4]:
df_num

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount
0,2763.519279,56274,69,32,5,0,1,384.811147
1,6979.535903,0,94,13,42,0,8,1131.464935
2,12887.431650,48767,108,18,38,0,2,566.472247
3,7645.861827,0,106,18,65,0,7,529.881344
4,2813.692575,43836,73,12,44,0,1,138.130879
...,...,...,...,...,...,...,...,...
9129,23405.987980,71941,73,18,89,0,2,198.234764
9130,3096.511217,21604,79,14,28,0,1,379.200000
9131,8163.890428,0,85,9,37,3,2,790.784983
9132,7524.442436,21941,96,34,3,0,3,691.200000


In [ ]:
for i in df_num.columns:
    plt.figure
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

In [5]:
df_des = df_num.describe().T

In [6]:
df_des['IQR'] = df_des['75%'] - df_des['25%']
df_des

,count,mean,std,min,25%,50%,75%,max,IQR
Customer Lifetime Value,9134.0,8004.940475,6870.967608,1898.007675,3994.251794,5780.182197,8962.167041,83325.381190,4967.915247
Income,9134.0,37657.380009,30379.904734,0.000000,0.000000,33889.500000,62320.000000,99981.000000,62320.000000
Monthly Premium Auto,9134.0,93.219291,34.407967,61.000000,68.000000,83.000000,109.000000,298.000000,41.000000
Months Since Last Claim,9134.0,15.097000,10.073257,0.000000,6.000000,14.000000,23.000000,35.000000,17.000000
Months Since Policy Inception,9134.0,48.064594,27.905991,0.000000,24.000000,48.000000,71.000000,99.000000,47.000000
Number of Open Complaints,9134.0,0.384388,0.910384,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
Number of Policies,9134.0,2.966170,2.390182,1.000000,1.000000,2.000000,4.000000,9.000000,3.000000
Total Claim Amount,9134.0,434.088794,290.500092,0.099007,272.258244,383.945434,547.514839,2893.239678,275.256594


In [7]:
df_des['lower_limit'] = df_des['25%'] - 1.5*df_des['IQR']
df_des['upper_limit'] = df_des['75%'] + 1.5*df_des['IQR']
df_des

,count,mean,std,min,25%,50%,75%,max,IQR,lower_limit,upper_limit
Customer Lifetime Value,9134.0,8004.940475,6870.967608,1898.007675,3994.251794,5780.182197,8962.167041,83325.381190,4967.915247,-3457.621076,16414.039912
Income,9134.0,37657.380009,30379.904734,0.000000,0.000000,33889.500000,62320.000000,99981.000000,62320.000000,-93480.000000,155800.000000
Monthly Premium Auto,9134.0,93.219291,34.407967,61.000000,68.000000,83.000000,109.000000,298.000000,41.000000,6.500000,170.500000
Months Since Last Claim,9134.0,15.097000,10.073257,0.000000,6.000000,14.000000,23.000000,35.000000,17.000000,-19.500000,48.500000
Months Since Policy Inception,9134.0,48.064594,27.905991,0.000000,24.000000,48.000000,71.000000,99.000000,47.000000,-46.500000,141.500000
Number of Open Complaints,9134.0,0.384388,0.910384,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000
Number of Policies,9134.0,2.966170,2.390182,1.000000,1.000000,2.000000,4.000000,9.000000,3.000000,-3.500000,8.500000
Total Claim Amount,9134.0,434.088794,290.500092,0.099007,272.258244,383.945434,547.514839,2893.239678,275.256594,-140.626647,960.399730


In [ ]:
df_des['lower_limit']['Customer Lifetime Value']

In [8]:
chop_list = ['Customer Lifetime Value','Monthly Premium Auto']

In [ ]:
data_defend_chop = data[(data[num_log_trans[0]] > ll_defend) & (data[num_log_trans[0]] < ul_defend)]


In [ ]:
def chop(x):
    ll = data['lower_limit'][i]
    ul = data['upper_limit'][i]
    if x > ul or x < ll:
        

In [9]:
def chop(lst,data, data_des):
    for i in lst:
        ll = data_des['lower_limit'][i]
        ul = data_des['upper_limit'][i]
        data_chop = data[(data[i] > ll) & (data[i] < ul)]
    return data_chop

In [10]:
new_df = chop(chop_list,df_num,df_des)

In [11]:
df_num.shape[0] - new_df.shape[0]

430

In [ ]:
def remove_outliers(col, df):
    flag = True
    constant = 1.5
    while flag:
        upper_limit = np.percentile(df[col],75) + constant*(np.percentile(df[col],75)-np.percentile(df[col],25))
        lower_limit = np.percentile(df[col], 25) - constant*(np.percentile(df[col], 75)-np.percentile(df[col], 25))
        filtered = data[(df[col]>lower_limit)  & (df[col]< upper_limit)]
        
        if (df.shape[0] - filtered.shape[0]) > 2000:
            constant += 0.5
        else:
            flag = False

    return filtered